# 載入基本環境(Basic Environment)

In [ ]:
import sys
print(sys.version)

In [ ]:
import os
import sys
import time
import requests as rs
import numpy as np
import pandas as pd
import feather as ft
import datetime as dt

# viz - matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams[u'font.sans-serif'] = ['simhei']
mpl.rcParams['axes.unicode_minus'] = False

# viz - others
import seaborn as sns
sns.set()
import chartify

# viz-interact
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# tqdm
from tqdm import tqdm 

# pandas_profiling
from pandas_profiling import ProfileReport

import warnings
warnings.filterwarnings('ignore')

# 讀取資料與探索資料(Load Data And EDA)

1. **視覺化**
    - 查看目標變量的分佈。當分佈不平衡時，根據評分標準和具體模型的使用不同，可能會嚴重影響性能。
    - **Numerical Variable**
        - 可以用Box Plot來直觀地查看它的分佈。
    - X**-Y Label Data**
        - 對於坐標類數據，可以用**Scatter Plot**來查看它們的分佈趨勢和是否有離群點的存在。
    - **Classification Data**
        - 對於分類問題，將數據根據Label 的不同著不同的顏色繪製出來，這對Feature 的構造很有幫助。
    - 繪製變量之間兩兩的分佈和相關度圖表。
    - [一個在著名的Iris數據集上做了一系列可視化的例子，非常有啟發性。](https://link.zhihu.com/?target=https%3A//www.kaggle.com/benhamner/d/uciml/iris/python-data-visualizations)

2. **統計檢定(Statistical Tests)**
    - 我們可以對數據進行一些統計上的測試來**驗證一些假設的顯著性**。
    - 雖然大部分情況下靠可視化就能得到比較明確的結論，但有一些定量結果總是更理想的。
    - 在某些比賽中，由於數據分佈比較奇葩或是噪聲過強，Public LB的分數可能會跟Local CV的結果相去甚遠。可以根據一些統計測試的結果來粗略地建立一個閾值，用來衡量一次分數的提高究竟是實質的提高還是由於數據的隨機性導致的。

# 識別問題(Identify Problems)
1. **識別是分類問題(classification problem)或迴歸問題(regression problem)**
    - 當label為二進制(binary values) → 分類問題(classification problem)。
        - Single column, binary values (classification problem, one sample belongs to one class only and there are only two classes)
        - Multiple column, binary values (classification problem, one sample belongs to one class, but there are more than two classes)
    - 當label為實數(real values) → 回歸問題(regression problem)。
        - Single column, real values (regression problem, prediction of only one value)
        - Multiple column, real values (regression problem, prediction of multiple values)

    - 當label不只一種時(multi label) → 分類問題(classification problem)
        - And multilabel (classification problem, one sample can belong to several classes)

2. **選擇對應的評估指標(Evaluation Metrics)**
    - 分類問題(classification problems)
        - skewed binary classification problem
            - ROC AUC or simply AUC
        - multi-label or multi-class classification problems
            - categorical cross-entropy
            - multiclass log loss
    - 回歸問題(regression problems)
        - mean squared error

# 前處理資料(Data Preprocessing)

- 項目
    1. 資料合併(merging)。
    2. 處理遺失值(missing value)。
    3. 處理離群值(outlier)。
    4. 處理重複值(duplicates)。
    5. 進行資料過篩(filtering)。 
    6. 資料儲存(storing)

- 參考資料
    - [[數據清洗]-Pandas 清洗"髒"數據（一）](https://hk.saowen.com/a/b8d38181b1f85660cc25830827acd63bf814aea8f3ea026f7e08fdcdd06c2089)
    - [Pythonic Data Cleaning With NumPy and Pandas - Real Python](https://realpython.com/python-data-cleaning-numpy-pandas/)
    - [Cleaning Data in Python](https://www.datacamp.com/courses/cleaning-data-in-python)

# 提取數據特徵(Identification of different variables in the data)

1. **確認資料型態**
    - 將數據轉化成模型需要的形式。
    - 數據有三種類型：
        - 數值(numerical variables)
        - 類別(categorical variables)
        - 文本(variables with text)
2. **轉換資料型態(Feature Encoding)**
    - 數值資料(numerical variables)
        - Separate out the numerical variables first. These variables don’t need any kind of processing and thus we can start applying normalization and machine learning models to these variables.
    - 類別資料(categorical variables)
        - There are two ways in which we can handle categorical data:
            - Convert the categorical data to labels

                ```python
                from sklearn.preprocessing import LabelEncoder
                ```

            - Convert the labels to binary variables (one-hot encoding)

                ```python
                # remember to convert categories to numbers first using LabelEncoder before applying OneHotEncoder on it.
                from sklearn.preprocessing import OneHotEncoder
                ```

        - 假設有一個Categorical Variable 一共有幾萬個取值可能，那麼創建Dummy Variables 的方法就不可行了。
        - 這時一個比較好的方法是根據Feature Importance 或是這些取值本身在數據中的出現頻率，為最重要（比如說前95% 的Importance）那些取值（有很大可能只有幾個或是十幾個）創建Dummy Variables，而所有其他取值都歸到一個“其他”類裡面。
        - 可參考：

            [Guide to Encoding Categorical Values in Python - Practical Business Python](http://pbpython.com/categorical-encoding.html)

    - 文本資料(text variables)
        - [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

            ```python
            from sklearn.feature_extraction.text import CountVectorizer
            ```

        - [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

            ```python
            from sklearn.feature_extraction.text import TfidfVectorizer
            ```

            - The TfidfVectorizer performs better than the counts most of the time and I have seen that the following parameters for TfidfVectorizer work almost all the time.

                ```python
                from sklearn.feature_extraction.text import TfidfVectorizer

                tfv = TfidfVectorizer(
                	min_df=3, max_features=None,
                	strip_accents='unicode', analyzer='word',
                	token_pattern=r'\w{1,}', ngram_range=(1, 2),
                	use_idf=1, smooth_idf=1, sublinear_tf=1, 
                	stop_words='english;)
                ```

            - If you are applying these vectorizers only on the training set, make sure to dump it to hard drive so that you can use it later on the validation set.

                ```python
                import cPickle
                cPickle.dump(vectorizer, open('vectorizer.pkl', 'wb'), -1)
                ```

        - 可參考：

            [文本数据预处理：sklearn 中 CountVectorizer、TfidfTransformer 和 TfidfVectorizer - U R MINE - CSDN博客](https://blog.csdn.net/m0_37324740/article/details/79411651)

# 切割訓練數據與測試數據(Split the data into two different parts)

1. **將原始資料分為訓練資料集(training data)與驗證資料集(validation data)**
    - Training Data用於訓練模型。
    - Validation Data用於檢驗這個模型的表現。

2. **依照問題類型選擇切分方法**
    - 分類問題(classification problems)

        ```python
        from sklearn.model_selection import StratifiedKFold
        ```

    - 回歸問題(regression problems)

        ```python
        from sklearn.model_selection import KFold
        ```
        
    - 簡單切
      ``` python 
        from sklearn.cross_validation import train_test_split
      ```

# 架設模型(Modeling)

# 評估模型(Evaluation Metrics)

1. **選擇對應的評估指標(Evaluation Metrics)**
    - 分類問題(classification problems)
        - skewed binary classification problem
            - [參考](https://taweihuang.hpd.io/2018/12/28/imbalanced-data-performance-metrics/)
            - ROC AUC or simply AUC
        - multi-label or multi-class classification problems
            - categorical cross-entropy
            - multiclass log loss
    - 回歸問題(regression problems)
        - mean squared error

2. **常用分類問題指標**
    | 混淆矩陣         | 「模型預測」為真 (positive) | 「模型預測」為非 (negative) | Rows Totals | 
    |------------------|-----------------------------|-----------------------------| --- |
    | 「真實情況」為真 | true positive (TP)          | false negative (FN)         |  P  |
    | 「真實情況」為非 | false positive (FP)         | true negative (TN)          | N  |

    - 混淆矩陣相關(Confusion Matrix)
      - [可參考](https://ithelp.ithome.com.tw/articles/10222697)
      - Accuracy: 
        - `Accuracy=[(TP)+(TN)]/(TP+TN+FP+FN)`
        - 最經典的顯示模型的正確度
      - Precision: 
        - `Precision=(TP)/(TP+FP)`
        - 我們比較重視「模型預測為真」的結果，是否能符合現實。真實為非(FN)」的影響相對比較還好。
      - Recall: 
        - `Recall=(TP)/(TP+FN)`
        - 我們比較重視「真實為真」的結果，模型能不能預測到。
      - F-Measure:
        - `F1 = 2*[(Precision*Recall)/(Precision+Recall)] = 2/(1/precision+1/recall)`
        - 自動化權衡Precision和Recall。
      - ROC曲線(Receiver Operating Characteristic)
        - [wiki介紹得很清楚](https://zh.wikipedia.org/wiki/ROC%E6%9B%B2%E7%BA%BF)
        - 縱軸 TPR(True Positive Rate): 
          - `TPR=(TP)/P=(TP)/(TP+FN)`
          - 真實為P且預測為P的比率
        - 橫軸 FPR(False Positive Rate): 
          - `FPR=(TP)/N=(TP)/(FP+TN)`
          - 真實為N但預測為P的比率
      - AUC(Area Under Curve)
        - [wiki介紹得很清楚](https://zh.wikipedia.org/wiki/ROC%E6%9B%B2%E7%BA%BF)
        - 簡單說：AUC值越大的分類器，正確率越高。
        - 從AUC判斷分類器（預測模型）優劣的標準：
          - AUC = 1，是完美分類器，採用這個預測模型時，存在至少一個閾值能得出完美預測。絕大多數預測的場合，不存在完美分類器。
          - 0.5 < AUC < 1，優於隨機猜測。這個分類器（模型）妥善設定閾值的話，能有預測價值。
          - AUC = 0.5，跟隨機猜測一樣（例：丟銅板），模型沒有預測價值。
          - AUC < 0.5，比隨機猜測還差；但只要總是反預測而行，就優於隨機猜測。
        - AUC of ROC是機器學習的社群最常使用來比較不同模型優劣的方法。然而近來這個做法開始受到質疑，因為有些機器學習的研究指出，AUC的雜訊太多，並且很常求不出可信又有效的AUC值（此時便不能保證AUC傳達本節開頭所述之意義），使得AUC在模型比較時產生的問題比解釋的問題更多
    - 交叉熵(cross-entropy)
      - [參考](https://medium.com/@chih.sheng.huang821/%E6%A9%9F%E5%99%A8-%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92-%E5%9F%BA%E7%A4%8E%E4%BB%8B%E7%B4%B9-%E6%90%8D%E5%A4%B1%E5%87%BD%E6%95%B8-loss-function-2dcac5ebb6cb)
- 損失函數(loss function)
  - MSE


- 梯度下降(gradient descent)
